In [0]:
df_media_customer_reviews = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/media_customer_reviews.parquet")
df_media_customer_reviews.write.mode("append").saveAsTable("media_customer_reviews_bronze")
df_media_customer_reviews.printSchema()

df_media_gold_reviews_chunked = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/media_gold_reviews_chunked.parquet")
df_media_gold_reviews_chunked.write.mode("append").saveAsTable("media_gold_reviews_chunked_bronze")
df_media_gold_reviews_chunked.printSchema()

df_sales_customers = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_customers.parquet")
df_sales_customers.write.mode("append").saveAsTable("sales_customers_bronze")
df_sales_customers.printSchema()

df_sales_franchises = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_franchises.parquet")
df_sales_franchises.write.mode("append").saveAsTable("sales_franchises_bronze")
df_sales_franchises.printSchema()

df_sales_suppliers = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_suppliers.parquet")
df_sales_suppliers.write.mode("append").saveAsTable("sales_suppliers_bronze")
df_sales_suppliers.printSchema()

df_sales_transactions = spark.read.parquet("/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_transactions.parquet")
df_sales_transactions.write.mode("append").saveAsTable("sales_transactions_bronze")
df_sales_transactions.printSchema()

root
 |-- review: string (nullable = true)
 |-- franchiseID: long (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- new_id: integer (nullable = true)

root
 |-- franchiseID: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- chunked_text: string (nullable = true)
 |-- chunk_id: string (nullable = true)
 |-- review_uri: string (nullable = true)

root
 |-- customerID: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- postal_zip_code: long (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- franchiseID: long (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = tru